In [8]:
import gc
import os
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import KMeansSMOTE,SMOTE,SVMSMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from catboost import CatBoostClassifier,CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer

In [9]:
train_data = [pd.read_csv(f'../input/g2net-prepare-features/train{i}.csv') for i in range(1,8)]
train_data = pd.concat(train_data)
train_data = train_data[train_data['target']!=-1]
train_data.reset_index(inplace = True, drop = True)
train_data.head()

,id,freq,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,...,ftr_151,ftr_152,ftr_153,ftr_154,ftr_155,ftr_156,ftr_157,ftr_158,ftr_159,target
0,001121a05,53.507778,111.014744,130.962706,698.178664,130.095403,140.503036,707.718994,133.564614,131.830009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,001121a05,53.508333,128.360798,122.289679,705.117086,115.351258,118.820468,633.130963,130.095403,124.024284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,001121a05,53.508889,108.412836,112.749350,668.690373,114.483955,104.943625,651.344319,112.749350,124.024284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,001121a05,53.509444,120.555074,109.280139,653.946227,116.218560,131.830009,714.657415,115.351258,129.228101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,001121a05,53.510000,117.085863,118.820468,679.965308,120.555074,133.564614,702.515178,116.218560,129.228101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [10]:
train_data.describe()

,freq,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,...,ftr_151,ftr_152,ftr_153,ftr_154,ftr_155,ftr_156,ftr_157,ftr_158,ftr_159,target
count,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,...,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000,216000.000000
mean,271.497179,118.811901,118.829358,676.631014,118.816179,118.807973,676.629037,118.814372,118.826740,118.832598,...,0.005012,0.061437,0.005258,0.005118,0.062298,0.004854,0.004990,0.005130,0.005095,0.666667
std,132.436566,10.107603,10.114644,22.728923,10.547393,10.532610,28.409118,10.100160,10.131544,10.532567,...,0.066209,0.233731,0.067715,0.066845,0.236580,0.065027,0.066235,0.066988,0.066746,0.471406
min,50.271667,76.378460,79.081633,586.252772,77.256944,69.267865,552.308344,77.362637,77.865267,74.346229,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,155.118889,111.912568,111.912568,661.410425,111.619795,111.651512,657.369663,111.937035,111.937035,111.639983,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,274.099444,118.676628,118.658641,676.354029,118.640398,118.629541,676.252723,118.666089,118.666089,118.655241,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,386.796667,125.521176,125.503498,691.555556,125.802879,125.802879,695.383256,125.435540,125.513958,125.813449,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,498.256111,247.041102,264.256510,1114.102847,245.484222,241.131665,1169.096844,230.308219,228.073993,246.354733,...,1.772657,4.303852,1.742160,1.750547,9.575626,1.771872,1.790510,1.834862,1.789309,1.000000


In [11]:
model_name = 'LGBM'
data_features = [i for i in train_data.columns if i not in ['id', 'target']]
skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
oof = np.zeros((len(train_data),))
data_predictions = []
test_indexes = []
model_list = []
for fold,(train_idx, valid_idx) in enumerate(skf.split(train_data, train_data['target'] )):
    X_train = train_data.loc[train_idx, data_features]
    y_train = train_data.loc[train_idx, 'target']
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    enn = EditedNearestNeighbours()
    X_train, y_train = enn.fit_resample(X_train, y_train)
    X_val = train_data.loc[valid_idx, data_features]
    y_val = train_data.loc[valid_idx, 'target']
    train_model = XGBClassifier(n_estimators = 10000) 
    train_model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=200, verbose = 100)
    joblib.dump(train_model,model_name+'_'+str(fold)+'.pkl')
    oof[valid_idx] = [i[1] for i in train_model.predict_proba(X_val)]
    model_list.append(train_model)

for i in range(1,81):
    test = pd.read_csv(f'../input/g2net-prepare-features/test{i}.csv')
    predictions = []
    test_indexes += test['id'].tolist()
    for train_model in model_list:
        predictions.append(train_model.predict_proba(test[data_features])[:,1])
    del test
    gc.collect()
    predictions =  np.average(predictions, axis = 0)
    data_predictions+=list(predictions)
    del predictions
    gc.collect()


#predictions = np.average(predictions, axis = 0)
np.save('oof.npy',oof)
waves_pred = pd.DataFrame(list(zip(test_indexes, data_predictions)), columns = ['id','target'])
waves_pred = waves_pred.groupby('id').mean().reset_index()
waves_pred.to_csv('submission.csv', index = False)
waves_pred.head()

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-logloss:0.77630
[100]	validation_0-logloss:0.60417
[200]	validation_0-logloss:0.41973
[300]	validation_0-logloss:0.35205
[400]	validation_0-logloss:0.31576
[500]	validation_0-logloss:0.30587
[600]	validation_0-logloss:0.31096
[693]	validation_0-logloss:0.31753


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-logloss:0.78734
[100]	validation_0-logloss:0.59971
[200]	validation_0-logloss:0.44625
[300]	validation_0-logloss:0.39617
[400]	validation_0-logloss:0.36629
[500]	validation_0-logloss:0.34784
[600]	validation_0-logloss:0.34666
[700]	validation_0-logloss:0.35737
[774]	validation_0-logloss:0.36963


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


...

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-logloss:0.78115
[100]	validation_0-logloss:0.56379
[200]	validation_0-logloss:0.40237
[300]	validation_0-logloss:0.33636
[400]	validation_0-logloss:0.31683
[500]	validation_0-logloss:0.30630
[600]	validation_0-logloss:0.31200
[700]	validation_0-logloss:0.32200
[715]	validation_0-logloss:0.32551


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-logloss:0.77582
[100]	validation_0-logloss:0.61305
[200]	validation_0-logloss:0.41937
[300]	validation_0-logloss:0.35505
[400]	validation_0-logloss:0.33464
[500]	validation_0-logloss:0.33338
[600]	validation_0-logloss:0.33270
[633]	validation_0-logloss:0.33610


,id,target
0,00054c878,0.027203
1,0007285a3,0.560320
2,00076c5a6,0.994152
3,001349290,0.009423
4,001a52e92,0.993607


In [12]:
waves_pred = pd.DataFrame(list(zip(set(test_indexes), data_predictions)), columns = ['id','target'])

In [13]:
waves_pred = waves_pred.groupby('id').mean().reset_index()
waves_pred.to_csv('submission.csv', index = False)

In [14]:
waves_pred

,id,target
0,00054c878,0.822101
1,0007285a3,0.626671
2,00076c5a6,0.003708
3,001349290,0.984633
4,001a52e92,0.002771
...,...,...
7970,ffbce04ef,0.001821
7971,ffc2d976b,0.648662
7972,ffc905909,0.980130
7973,ffe276f3e,0.294429
